<img src="../../images/banners/python-advanced.png" width="600"/>

# <img src="../../images/logos/python.png" width="23"/> Speed Up Your Python Program With Concurrency 


## <img src="../../images/logos/toc.png" width="20"/> Table of Contents 
* [What Is Concurrency?](#what_is_concurrency?)
* [What Is Parallelism?](#what_is_parallelism?)
* [When Is Concurrency Useful?](#when_is_concurrency_useful?)
* [How to Speed Up an I/O-Bound Program](#how_to_speed_up_an_i/o-bound_program)
* [Synchronous Version](#synchronous_version)
    * [Why the Synchronous Version Rocks](#why_the_synchronous_version_rocks)
    * [The Problems With the Synchronous Version](#the_problems_with_the_synchronous_version)
* [`threading` Version](#`threading`_version)

---

<a class="anchor" id="what_is_concurrency?"></a>
## What Is Concurrency?

The dictionary definition of concurrency is simultaneous occurrence. In Python, the things that are occurring simultaneously are called by different names (thread, task, process) but at a high level, they all refer to a sequence of instructions that run in order.

I like to think of them as different trains of thought. Each one can be stopped at certain points, and the CPU or brain that is processing them can switch to a different one. The state of each one is saved so it can be restarted right where it was interrupted.

You might wonder why Python uses different words for the same concept. It turns out that threads, tasks, and processes are only the same if you view them from a high level. Once you start digging into the details, they all represent slightly different things. You’ll see more of how they are different as you progress through the examples.

Now let’s talk about the simultaneous part of that definition. You have to be a little careful because, when you get down to the details, only `multiprocessing` actually runs these trains of thought at literally the same time. Threading and `asyncio` both run on a single processor and therefore only run one at a time. They just cleverly find ways to take turns to speed up the overall process. Even though they don’t run different trains of thought simultaneously, we still call this concurrency.

The way the threads or tasks take turns is the big difference between threading and asyncio. In threading, the operating system actually knows about each thread and can interrupt it at any time to start running a different thread. This is called pre-emptive multitasking since the operating system can pre-empt your thread to make the switch.

Pre-emptive multitasking is handy in that the code in the thread doesn’t need to do anything to make the switch. It can also be difficult because of that “at any time” phrase. This switch can happen in the middle of a single Python statement, even a trivial one like `x = x + 1`.

`Asyncio`, on the other hand, uses cooperative multitasking. The tasks must cooperate by announcing when they are ready to be switched out. That means that the code in the task has to change slightly to make this happen.

The benefit of doing this extra work up front is that you always know where your task will be swapped out. It will not be swapped out in the middle of a Python statement unless that statement is marked. You’ll see later how this can simplify parts of your design.

<a class="anchor" id="what_is_parallelism?"></a>
## What Is Parallelism?

So far, you’ve looked at concurrency that happens on a single processor. What about all of those CPU cores your cool, new laptop has? How can you make use of them? multiprocessing is the answer.

With `multiprocessing`, Python creates new processes. A process here can be thought of as almost a completely different program, though technically they’re usually defined as a collection of resources where the resources include memory, file handles and things like that. One way to think about it is that each process runs in its own Python interpreter.

Because they are different processes, each of your trains of thought in a multiprocessing program can run on a different core. Running on a different core means that they actually can run at the same time, which is fabulous. There are some complications that arise from doing this, but Python does a pretty good job of smoothing them over most of the time.

Now that you have an idea of what concurrency and parallelism are, let’s review their differences, and then we can look at why they can be useful:

|Concurrency Type|Switching Decision | Number of Processors|
|:--|:--|:--|
|Pre-emptive multitasking (threading)|The operating system decides when to switch tasks external to Python|1|
|Cooperative multitasking (asyncio)|The tasks decide when to give up control.|1|
|Multiprocessing (multiprocessing)|The processes all run at the same time on different processors.|Many|

Each of these types of concurrency can be useful. Let’s take a look at what types of programs they can help you speed up.

<a class="anchor" id="when_is_concurrency_useful?"></a>
## When Is Concurrency Useful?

Concurrency can make a big difference for two types of problems. These are generally called CPU-bound and I/O-bound.

I/O-bound problems cause your program to slow down because it frequently must wait for input/output (I/O) from some external resource. They arise frequently when your program is working with things that are much slower than your CPU.

Examples of things that are slower than your CPU are legion, but your program thankfully does not interact with most of them. The slow things your program will interact with most frequently are the file system and network connections.

Let’s see what that looks like:

<img src="../images/iobound.png" alt="iobound" width=500 align="center" />

In the diagram above, the blue boxes show time when your program is doing work, and the red boxes are time spent waiting for an I/O operation to complete. This diagram is not to scale because requests on the internet can take several orders of magnitude longer than CPU instructions, so your program can end up spending most of its time waiting. This is what your browser is doing most of the time.

On the flip side, there are classes of programs that do significant computation without talking to the network or accessing a file. These are the CPU-bound programs, because the resource limiting the speed of your program is the CPU, not the network or the file system.

Here’s a corresponding diagram for a CPU-bound program:

<img src="../images/cpubound.png" alt="iobound" width=500 align="center" />

As you work through the examples in the following section, you’ll see that different forms of concurrency work better or worse with CPU-bound and I/O-bound programs. Adding concurrency to your program adds extra code and complications, so you’ll need to decide if the potential speed up is worth the extra effort. By the end of this article, you should have enough info to start making that decision.

Here’s a quick summary to clarify this concept:

|I/O-Bound Process|CPU-Bound Process|
|:--|:--|
|Your program spends most of its time talking to a slow device, like a network connection, a hard drive, or a printer.|You program spends most of its time doing CPU operations.|
|Speeding it up involves overlapping the times spent waiting for these devices.|Speeding it up involves finding ways to do more computations in the same amount of time.|

You’ll look at I/O-bound programs first. Then, you’ll get to see some code dealing with CPU-bound programs.

<a class="anchor" id="how_to_speed_up_an_i/o-bound_program"></a>
## How to Speed Up an I/O-Bound Program

Let’s start by focusing on I/O-bound programs and a common problem: downloading content over the network. For our example, you will be downloading web pages from a few sites, but it really could be any network traffic. It’s just easier to visualize and set up with web pages.

<a class="anchor" id="synchronous_version"></a>
## Synchronous Version
We’ll start with a non-concurrent version of this task. Note that this program requires the requests module. You should run pip install requests before running it, probably using a virtualenv. This version does not use concurrency at all:

In [ ]:
import requests
import time

In [ ]:
def download_site(url, session):
    with session.get(url) as response:
        print(f"Read {len(response.content)} from {url}")


def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            download_site(url, session)

In [ ]:
sites = [
    "https://www.jython.org",
    "http://olympus.realpython.org/dice",
] * 80
start_time = time.time()
download_all_sites(sites)
duration = time.time() - start_time
print(f"Downloaded {len(sites)} in {duration} seconds")

As you can see, this is a fairly short program. `download_site()` just downloads the contents from a URL and prints the size. One small thing to point out is that we’re using a Session object from requests.

It is possible to simply use `get()` from requests directly, but creating a `Session` object allows requests to do some fancy networking tricks and really speed things up.

`download_all_sites()` creates the Session and then walks through the list of sites, downloading each one in turn. Finally, it prints out how long this process took so you can have the satisfaction of seeing how much concurrency has helped us in the following examples.

The processing diagram for this program will look much like the I/O-bound diagram in the last section.

>‌ **Note:** Network traffic is dependent on many factors that can vary from second to second. I’ve seen the times of these tests double from one run to another due to network issues.

<a class="anchor" id="why_the_synchronous_version_rocks"></a>
### Why the Synchronous Version Rocks

The great thing about this version of code is that, well, it’s easy. It was comparatively easy to write and debug. It’s also more straight-forward to think about. There’s only one train of thought running through it, so you can predict what the next step is and how it will behave.

<a class="anchor" id="the_problems_with_the_synchronous_version"></a>
### The Problems With the Synchronous Version

The big problem here is that it’s relatively slow compared to the other solutions we’ll provide.

Being slower isn’t always a big issue, however. If the program you’re running takes only 2 seconds with a synchronous version and is only run rarely, it’s probably not worth adding concurrency. You can stop here.

What if your program is run frequently? What if it takes hours to run? Let’s move on to concurrency by rewriting this program using threading.

<a class="anchor" id="`threading`_version"></a>
## `threading` Version
As you probably guessed, writing a threaded program takes more effort. You might be surprised at how little extra effort it takes for simple cases, however. Here’s what the same program looks like with threading: